# Intro to LangChain

- Using language models
- Using PromptTemplates and OutputParsers
- Using LangChain Expression Language (LCEL) to chain components together
- Debugging and tracing your application using LangSmith

## Install dependencies

Some providers come in their own package, such as groq and openai.

In [ ]:
%pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.6/974.6 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.8/315.8 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.3/125.3 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 5.2 MB/s eta 0:00:00


In [ ]:
%pip install -qU langchain-openai
%pip install -qU langchain-groq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.5/103.5 kB 2.9 MB/s eta 0:00:00


## Runnables

`ChatOpenAI`is an instance of a `Runnable` in Langchain. It is similar to the client we created in the previous version, but it is much more flexible.

A `Runnable` is an object that exposes a standard interface for interacting with it.

This means that `Runnables` have the same methods that can be called. This makes working with the framework much more flexible since they allow you to work with several service providers.

In [ ]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

from langchain_openai import ChatOpenAI

model_openai = ChatOpenAI() # model defaults to 'gpt-3.5-turbo'

··········


In [ ]:
import getpass
import os

os.environ["GROQ_API_KEY"] = getpass.getpass()

from langchain_groq import ChatGroq

model_groq = ChatGroq(model="llama3-8b-8192")

··········


In [ ]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content="Translate the following from French into English"),
    HumanMessage(content="Bonjour ! Il fait très beau à Paris aujourd'hui."),
]

In [ ]:
response_openai = model_openai.invoke(messages)

response_groq = model_groq.invoke(messages)

In [ ]:
response_openai

AIMessage(content='Hello! The weather is very nice in Paris today.', response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 29, 'total_tokens': 40}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-c8ae9b50-52fb-4416-b468-607a9e2cc4c9-0', usage_metadata={'input_tokens': 29, 'output_tokens': 11, 'total_tokens': 40})

In [ ]:
response_groq

AIMessage(content='Here is the translation:\n\n"Hello! It\'s very nice in Paris today."', response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 33, 'total_tokens': 50, 'completion_time': 0.012881826, 'prompt_time': 0.005488198, 'queue_time': None, 'total_time': 0.018370024}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_af05557ca2', 'finish_reason': 'stop', 'logprobs': None}, id='run-f5c87ad9-b570-4629-902b-e109223ee6ca-0')

## Output Parsers

In a process, a LLM call will only be one of the steps that compose it. So it is a good idea to always have a consistent data type as a result.

In this case, we are getting an `AIMessage` instance as a result from calling our model. Let's use `StrOutputParser`, an Output Parser to makes sure your result is always a string.

In [ ]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

In [ ]:
parser.invoke(response_groq)

'Here is the translation:\n\n"Hello! It\'s very nice in Paris today."'

In [ ]:
parser.invoke(response_openai)

'Hello! The weather is very nice in Paris today.'

## Chains

We can now chain these two components together. This way we can have the output parser be applied to the model call every single time.

To do this, we can create a `chain`.

Similarly to bash, chains in LangChain are created using the `|` operator.

In [ ]:
model = model_groq | parser

chain = model | parser

In [ ]:
# it exposes the same interface as the model and parser

chain.invoke(messages)

'Here\'s the translation:\n\n"Hello! It\'s very beautiful in Paris today."'

## Prompt Templates

Prompt templates allow you to streamline a consistent prompt that you will send to your LLM. You will only need to specify in your call the variables that you want to change.

In [ ]:
# prompt templates

from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template("Tell me a joke about {topic}")

prompt_template.invoke({"topic": "cats"})

In [ ]:
# chat prompt templates

from langchain_core.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", "Translate the following into {language}:"),
        ("user", "{text}")
    ]
)

In [ ]:
result = prompt_template.invoke({"language": "italian", "text": "hi"})

result

ChatPromptValue(messages=[SystemMessage(content='Translate the following into italian:'), HumanMessage(content='hi')])

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage

prompt_template = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant"),
    MessagesPlaceholder("msgs")
])

prompt_template.invoke({"msgs": [HumanMessage(content="hi!")]})

That's a `ChatPromptValue`. Let's turn that into messages:

In [ ]:
result.to_messages()

[SystemMessage(content='Translate the following into italian:'),
 HumanMessage(content='hi')]

## Chaining all together

In [ ]:
chain = prompt_template | model | parser

In [ ]:
chain.invoke({"language": "french", "text": "Hello! This is a translated sentence."})

'Bonjour ! Voici une phrase traduite.'

## More on Runnables

We will see streaming in detail later. But for now, here is a sneak peak of what streaming looks like in LangChain.

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

import nest_asyncio

nest_asyncio.apply()

prompt = ChatPromptTemplate.from_template("tell me a story about {topic}")
parser = StrOutputParser()
chain = prompt | model_groq | parser

async for chunk in chain.astream({"topic": "parrot"}):
    print(chunk, end="", flush=True)

Here's a story about a colorful and charming parrot:

In a lush jungle, surrounded by vibrant flowers and towering trees, there lived a majestic parrot named Sunny. Sunny was a brilliant shade of blue and green, with an iridescent sheen that shimmered in the sunlight. Her feathers were so bright that they seemed to glow from within, making her a beacon of joy and beauty in the jungle.

Sunny loved to fly through the jungle, exploring every nook and cranny, and making friends with all the creatures she met along the way. She was a gentle soul, with a kind heart and a quick wit. Her best friend was a wise old owl named Professor Hootenanny, who lived in a hollow tree near the jungle's central clearing.

One day, a severe storm rolled in over the jungle, bringing heavy rains and strong winds. The creatures of the jungle huddled together, frightened by the loud thunder and flashes of lightning. But Sunny, being the brave parrot she was, decided to take matters into her own beak. She flew o